In [ ]:
import os
import pandas as pd

# Path to the folder inside "logs"
folder_path = os.path.join("logs_archive", "run_20250505_030057")


# Create a list to store file info
file_info = []

# Loop through each file and get row counts
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        try:
            with open(file_path) as f:
                row_count = sum(1 for _ in f) - 1  # subtract 1 for header
            file_info.append({"filename": filename, "row_count": row_count})
        except Exception as e:
            file_info.append({"filename": filename, "row_count": f"Error: {e}"})

# Convert to DataFrame
row_counts_df = pd.DataFrame(file_info)

# Display the DataFrame
row_counts_df



,filename,row_count
0,avg-sigma.csv,1107
1,fillin-pass.csv,4835
2,pcg-resid.csv,4835
3,line-search.csv,4835
4,max-search-dir.csv,4835
5,dyn-consumed.csv,4835
6,toi-recale.csv,4835
7,initialize.csv,1
8,dt.csv,1107
9,num-contact.csv,4835


In [ ]:

# around row 641 for csvs with 1107 rows is when dt might have changed 
# Load the CSV
df = pd.read_csv("logs_archive/run_20250505_030057/newton-steps.csv")
df['time'] = df['time_or_frame'].copy()
df['newton_steps'] = df['value'].copy()

# Calculate the difference between consecutive vid_time values
df['vid_time_diff'] = df['time'].diff()

# Find the first index where the increment is (roughly) 0.02
threshold = 0.015  # in case it's slightly off from 0.02 due to float rounding
switch_row = df[df['vid_time_diff'] > threshold].index[0]

# Show the row and maybe a few before/after for context
df.iloc[switch_row - 2 : switch_row + 3]
df


,time_or_frame,value,time,newton_steps,vid_time_diff
0,0.000000,3,0.000000,3,NaN
1,0.000227,2,0.000227,2,0.000227
2,0.000507,1,0.000507,1,0.000280
3,0.000694,2,0.000694,2,0.000187
4,0.000883,3,0.000883,3,0.000189
...,...,...,...,...,...
1102,3.968071,1,3.968071,1,0.019976
1103,3.975487,1,3.975487,1,0.007416
1104,3.983251,1,3.983251,1,0.007764
1105,3.990983,1,3.990983,1,0.007732


In [ ]:
# this successfully maps frames to newton steps!!!!


# Load both CSVs
newton_df = pd.read_csv("logs_archive/run_20250505_030057/newton-steps.csv")
frame_df = pd.read_csv("logs_archive/run_20250505_030057/frame-to-time.csv")
newton_df['time'] = newton_df['time_or_frame'].copy()
newton_df['newton_steps'] = newton_df['value'].copy()
newton_df = newton_df[['time','newton_steps']]

frame_df['frame'] = frame_df['time_or_frame'].copy()
frame_df['time']=frame_df['value'].copy()
frame_df = frame_df[['frame','time']]
frame_df = frame_df.sort_values("time").reset_index(drop=True)


# Merge on the closest matching time using merge_asof
frame_df_sorted = frame_df.sort_values("time")
newton_df_sorted = newton_df.sort_values("time")

# Map time to closest frame time
merged_df = pd.merge_asof(newton_df_sorted, frame_df_sorted, on="time", direction='backward')

# Rename for clarity
merged_df = merged_df.rename(columns={"frame": "corresponding_frame"})

# Optional: see the result
merged_df.head()


,time,newton_steps,corresponding_frame
0,0.000000,3,0
1,0.000227,2,0
2,0.000507,1,0
3,0.000694,2,0
4,0.000883,3,0


In [ ]:
newton_df_sorted